In [ ]:

#not uisng skip to next script 

'''
import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
#final = pd.read_csv("inputs/deduplicated_CIHI.csv", low_memory=False)
final = pd.read_csv("inputs/deduplicated_CSD.csv", low_memory=False)

#read dedupe file
df = pd.read_csv("outputs/probable_duplicates_PROV.csv")

def hierarchy(id1, id2): 

    try:

        
        check_file1 = final.loc[final.idx == id1].filename.item()
        check_file2 = final.loc[final.idx == id2].filename.item()
        check_name1 = final.loc[final.idx == id1].facility_name.item()
        check_name2 = final.loc[final.idx == id2].facility_name.item()

        filename_priority = ['QC_healthcare_facilities.csv', 'QC_Gatineau_public_places_including_hospitals,csv', 'QC_Quebec_City_public_places.csv', 
                             'ON_Ottawa_hospital.csv', 'ON_Durham_healthcare_facilities.csv', 'ON_healthcare_facilities.csv', 'NB_hospital_horizon.csv', 'NB_nursing_homes.csv']

        facilityname_priority = ['hospital', 'hôpita','hopita']

        if filename_priority.count(check_file1) == 1:

            if filename_priority.count(check_file2) == 1:

                priority1 = filename_priority.index(check_file1)
                priority2 = filename_priority.index(check_file2)

                if priority1 > priority2:
                    return id1
                elif priority1 < priority2:
                    return id2
                elif priority1 == priority2:

                    if any(substring in check_name1.lower() for substring in facilityname_priority):
                        return id2
                    elif any(substring in check_name2.lower() for substring in facilityname_priority):
                        return id1
                    else:
                        return id2

            elif filename_priority.count(check_file2) == 0:
                return id2

        elif filename_priority.count(check_file1) == 0:

            if filename_priority.count(check_file2) == 1:
                return id1

            elif filename_priority.count(check_file2) == 0:

                if check_file1 == 'CIHI_healthcare_facilities.csv' and check_file2 != 'CIHI_healthcare_facilities.csv':
                    return id1

                elif check_file2 == 'CIHI_healthcare_facilities.csv' and check_file1 != 'CIHI_healthcare_facilities.csv':
                    return id2

                else:

                    if check_beds1 == np.nan and check_beds2 == np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2

                    elif check_beds1 == np.nan and check_beds2 != np.nan:
                        return id1

                    elif check_beds1 != np.nan and check_beds2 == np.nan:
                        return id2

                    elif check_beds1 != np.nan and check_beds2 != np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2



    except ValueError:
        return "Item not found."


df["to_remove"] = df.apply(lambda x: hierarchy( id1 = x.idx1, id2 = x.idx2), axis = 1)
#df.to_csv('test.csv')


''' 


In [6]:
import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_CIHI.csv", low_memory=False)
#final = pd.read_csv("inputs/deduplicated_CSD.csv", low_memory=False)

#read dedupe file
#df = pd.read_csv("outputs/probable_duplicates_PROV.csv")
df = pd.read_csv("outputs/pairs_PROV.csv", low_memory=False)


def check_1(address, distance):

    if distance != None:
        if address >= 0.9 and distance < 2:
            return True
        else:
            return False

    else:
        return False

df["Check_1"] = df.apply(lambda x: check_1(address = x.Addr_CS, 
                          distance = x.Distance), axis=1)

#df.to_csv('test.csv')

In [7]:
def check_2(address, distance, name):

    if distance != None:
        if address >= 0.90 and distance < 1 and name > 0.85:
            return True
        else:
            return False

    else:
        return False

df["Check_2"] = df.apply(lambda x: check_2(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS), axis=1)

In [8]:
def check_3(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.9 and distance < 1 and name > 0.93 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_3"] = df.apply(lambda x: check_3(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

#df.to_csv('test.csv')

In [9]:
# remove duplicates

rows = df.index[(df['Check_1'] == True) | (df['Check_2'] == True) | (df['Check_3'] == True)].tolist()
false_rows = df.index[(df['Check_1'] == False) & (df['Check_2'] == False) | (df['Check_3'] == True)].tolist()

df_true = df.loc[rows]
df_false = df.loc[false_rows]

df_true.to_csv('outputs/duplicates_PROV.csv', index=False)
df_false.to_csv('outputs/NOT_duplicates_PROV.csv', index=False)

hashes = df_true #['to_remove'].tolist()

final = final[~final['idx'].isin(hashes)]
final.to_csv('deduplicated.csv', index=False)
#final.to_csv('../8-FinalCleaning/deduplicated.csv', index=False)